In [1]:
import sys
sys.path.append('..') # to make modules importable

In [2]:
import pickle
from tqdm.notebook import tqdm
import pandas as pd
from dicts import var_vocab, args, topic_to_russian
from classes.Preprocessor import Preprocessor

In [3]:
with open("../data/gpt_scores.pkl", "rb") as f:
    res = pickle.load(f)

In [4]:
best_model = None
best_score = 0
for model, score in res.items():
    if best_score < score:
        best_score = score
        best_model = model

In [5]:
print(f"{best_model} score is {best_score}")

epoch_5_num_52_loss_0.7315569369028031 score is 1179


In [6]:
df = pd.read_csv("../data/df.csv", low_memory=False)
df.sample(3)

,document.id,source,stage,source_text,lemm_text,city,region,date,data,assessor,...,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id,text_sentiment
28430,973519953,iqbuzz,2,"Или: - Стоять, баклажан! (в иной интерпретации...",re улыбнуться стоять баклажан иной интерпретац...,Москва,Москва,2015-07-20,2016-09-27 03:52:46,adzhigitova,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,черножопый_67,NaN
12047,805226841,iqbuzz,2,"крымчане это крымские татары,караимы и прочие ...",григорий крымчанин крымский прочий коренной на...,Санкт-Петербург,Санкт-Петербург,2015-03-13,2016-10-03 23:05:42,yuvkhudyakov,...,no,3.0,irrel,3.0,irrel,1.0,no,"крымчанин, русский, чех, германец",караим_106,-1.0
59848,875118570,iqbuzz,3,Нацизм? Караул шо творится ! Лично мне не важн...,нацизм караул шо твориться лично важный какой ...,Хабаровск,Хабаровский край,2015-04-25,2017-03-22 23:13:31,adzhigitova,...,unk,3.0,irrel,3.0,irrel,1.0,no,NaN,875118570,-1.0


In [7]:
ids = df['document.id'].sample(20)

In [8]:
preprocessor = Preprocessor(df=df, args=args, var_vocab=var_vocab, topic_to_russian=topic_to_russian)

In [9]:
from torch.utils.data import DataLoader
from gpt_classes.GPTDataset import GPTDataset
from gpt_classes.GPTModel import GPTModel
import torch

In [10]:
dataset = GPTDataset(df[df['document.id'].isin(ids)], args=args, preprocessor=preprocessor)

In [11]:
loader = DataLoader(dataset, batch_size=16)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = GPTModel(device).to(device)

In [12]:
model.load_state_dict(torch.load(f"../models/{best_model}", map_location=torch.device(device)))

<All keys matched successfully>

In [13]:
all_generated = []
all_texts = []
all_desc = []

In [14]:
with torch.no_grad():
    for i, (data, cut_data) in enumerate(tqdm(loader), 1):
        outputs = model(data, cut_data)

        for el in data:
            text, descr = el.split("Описание: ", 1)
            all_texts.append(text)
            all_desc.append(descr)

        generated = model.my_generate(cut_data)
        all_generated.extend(generated)
    

  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
for gen, text, descr in zip(all_generated, all_texts, all_desc):
    print(f"\n{gen}\n======\n")


Задание: Сгенерируй описание следующего текста, оцени тональность текста, оцени наличие этнического конфликта, наличие позитивного взаимодействия этничностей, мнение о алтаец в тексте, является ли этничность алтаец доминирующей, является ли этничность алтаец агрессором, является ли этничность алтаец опасной.
Текст: В России нельзя выйти на улицу с факелами ночью и скандировать «смерть бурятам», «алтайцы-чемодан,вокзал Монголия»,за это в России садят в тюрьму. Особо рьяных-надолго. И никакие оправдания, типа: «мы же ни одного бурята не убили»,от срока не спасут. А на Украине можно… И если лично вы против,но ваше мнение ничего не решает в вашей стране,то почему оно должно что-то значить в другой? Может в этом и есть причина нынешней российской нелюбви к украинцам?! Господа украинцы,вы не пробовали поискать причину в себе, прежде,чем обвинять других?!
Описание: этот текст является негативным, в текст есть этнический конфликт, этот автор не является опасной
СвернутьНовости от 17.02.2015 -